In [0]:
%python
print("hello")
print("hi")

In [0]:
describe raw_resume_data

In [0]:
select * from raw_resume_data limit 10

In [0]:
with exp_skill as (
  select resume_id, name, job_title, skills, location, date, experience_years, skill
  from raw_resume_data 
  lateral view explode(split(skills, ' ')) as skill
)
select resume_id,name,skill from exp_skill limit 10

In [0]:
with exp_skill as (
  select resume_id, name, job_title, skills, location, date, experience_years, skill
  from raw_resume_data 
  lateral view explode(split(skills, ' ')) as skill
)
select job_title,skill,count(skill) as total_skills from exp_skill group by job_title,skill order by total_skills desc limit 10

In [0]:
%python
df = spark.table('raw_resume_data')
df.show(10)

In [0]:
%python
from pyspark.sql.functions import col, when
df1 = df.withColumn("job_title",
    when(col("job_title") == "Data Engr.", "Data Engineer")
    .when(col("job_title") == "Software Dev", "Software Developer")
    .when(col("job_title") == "Proj. Manager", "Project Manager")
    .when(col("job_title") == "Biz Analyst", "Business Analyst")
    .when(col("job_title") == "Cloud Engr.", "Cloud Engineer")
    .when(col("job_title") == "DevOps Eng", "DevOps Engineer")
    .when(col("job_title") == "ML Engr.", "Machine Learning Engineer")
    .when(col("job_title") == "Fullstack Dev", "Full Stack Developer")
    .when(col("job_title") == "Cybersecurity Spec.", "Cybersecurity Analyst")
    .otherwise(col("job_title"))
)
display(df1)

In [0]:
%python
# Define location mapping
location_mapping = {
    "NY, USA": "New York, USA",
    "San Fran, US": "San Francisco, USA",
    "London, United Kingdom": "London, UK",
    "Berlin-Germany": "Berlin, Germany",
    "Toronto-CA": "Toronto, Canada",
    "Sydney/AUS": "Sydney, Australia",
    "Mumbai, IND": "Mumbai, India",
    "Singpore": "Singapore, Singapore",
    "Tokyo-JP": "Tokyo, Japan",
    "Paris FR": "Paris, France"
}

# Create a UDF to map locations
from pyspark.sql.functions import udf, col, split
from pyspark.sql.types import StringType

def map_location(location):
    return location_mapping.get(location, location)

map_location_udf = udf(map_location, StringType())

# Apply location normalization
df = df.withColumn("location", map_location_udf(col("location")))

# Split location into city and country
df = df.withColumn("city", split(col("location"), ", ")[0]) \
       .withColumn("country", split(col("location"), ", ")[1])

# Drop the original location column
df1 = df.drop("location")

# Display the resulting DataFrame
display(df1)

In [0]:
%python
# Set the configuration to bypass the ANSI mode error
spark.conf.set("spark.sql.ansi.enabled", "false")

from pyspark.sql.functions import col, to_date, when, date_format

date_formats = [
    "MM-dd-yyyy", "yyyy/MM/dd", "dd MMM, yyyy", "MMM dd yyyy",
    "yyyy-dd-MM", "dd-MM-yyyy", "yyyy.MM.dd", "MM/dd/yyyy",
    "dd/MM/yy", "MMM/dd/yyyy"
]

df = spark.table("workspace.default.raw_resume_data")

df = df.withColumn("normalized_date", to_date(col("date"), "MM-dd-yyyy"))

for fmt in date_formats:
    df = df.withColumn("normalized_date", 
                       when(col("normalized_date").isNull(), to_date(col("date"), fmt))
                       .otherwise(col("normalized_date")))

df = df.withColumn("final_date", date_format(col("normalized_date"), "yyyy-MM-dd")) \
       .drop("date", "normalized_date") \
       .withColumnRenamed("final_date", "date")

display(df)

In [0]:
%python
df = df.drop('location')

In [0]:
%python
from pyspark.sql.functions import col, when
df1 = df.withColumn("job_title",
    when(col("job_title") == "Data Engr.", "Data Engineer")
    .when(col("job_title") == "Software Dev", "Software Developer")
    .when(col("job_title") == "Proj. Manager", "Project Manager")
    .when(col("job_title") == "Biz Analyst", "Business Analyst")
    .when(col("job_title") == "Cloud Engr.", "Cloud Engineer")
    .when(col("job_title") == "DevOps Eng", "DevOps Engineer")
    .when(col("job_title") == "ML Engr.", "Machine Learning Engineer")
    .when(col("job_title") == "Fullstack Dev", "Full Stack Developer")
    .when(col("job_title") == "Cybersecurity Spec.", "Cybersecurity Analyst")
    .otherwise(col("job_title"))
)
display(df1)


In [0]:
%python
df1.createOrReplaceTempView("processed_resume_data")

In [0]:
describe processed_resume_data

In [0]:
with exp_skill as (
  select resume_id, name, job_title, skills, city,country as location, date, experience_years, explode(split(skills, ' ')) as skill 
  from processed_resume_data
)
select * from exp_skill limit 10

In [0]:
select job_title, skill
from processed_resume_data 
lateral view explode(split(skills, ' ')) as skill 
group by job_title, skill 
order by count(skill) desc 
limit 10

In [0]:
select job_title, city, count(job_title) as total_jobs 
from processed_resume_data 
group by job_title,city
order by total_jobs desc 
limit 10

In [0]:
select *, rank() over (partition by job_title order by experience_years desc) as rnk 
from processed_resume_data

In [0]:
SELECT city, job_title,COUNT(job_title) AS count_jobs 
FROM processed_resume_data 
GROUP BY city,job_title
ORDER BY count_jobs DESC 
LIMIT 10

In [0]:
select city,skill,count(skill) as total_skills from processed_resume_data lateral view explode(split(skills, ' ')) as skill  group by city,skill order by total_skills desc limit 10 